Imports

In [1]:
import utils.scrape_no_js as bourso
import utils.inputs as inputs
import utils.helper_functions as hf

from datetime import datetime, date, timedelta
import pandas as pd
import numpy as np
import yfinance as yf
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
np.set_printoptions(threshold=np.inf)

Config

In [2]:
buy_time = 'Open'
loss_limit = 0.997

cumulated_probs_target = 0.9

fee = 0.002

download_days = 500 #should be > 151 = 90 (input days) * 7/5 (days per week) * 1.2 (margin in case of vacations)

model_path = './outputs/classifier_model.keras'
scaler_path = './outputs/scaler.pkl'

In [3]:
tickers_file_path = './db/tickers_euronext_regulated_euro_500k€.xlsx'
df_tickers = pd.read_excel(tickers_file_path)

Download past data

In [4]:
def get_past_date(days_past):
    today = date.today()
    past_date = today - timedelta(days=days_past)
    formatted_date = past_date.strftime('%Y-%m-%d')
    return formatted_date

def days_difference(date1, date2): # can handle date objects and 'YYYY-MM-DD' strings
    date1 = pd.to_datetime(date1)
    date2 = pd.to_datetime(date2)

    difference = abs((date2 - date1).days)
    return difference

In [5]:
start_date = get_past_date(download_days)
today = get_past_date(0)

print(f'start download date: {start_date}')
print(f'today: {today}')
print(f'number of tickers: {len(df_tickers)}')


start download date: 2023-03-04
today: 2024-07-16
number of tickers: 870


In [6]:
yahoo_tickers = df_tickers['yahoo'].iloc[1:872].values.tolist()

download_past_data = yf.download(yahoo_tickers, start=start_date, end=today)
df_download_past = pd.DataFrame(download_past_data)
df_download_past_filled = hf.fillnavalues(df_download_past)
# df_download = df_download.dropna(axis='columns', how='all') #remove columns with only nan values

print(df_download_past_filled.tail(3).to_markdown())


[*********************100%%**********************]  869 of 869 completed

3 Failed downloads:
['BNJW.AS', 'NAIW.AS', 'WTFIN.MI']: Exception('%ticker%: No timezone found, symbol may be delisted')


| Date                |   ('Adj Close', 'A2A.MI') |   ('Adj Close', 'A5G.IR') |   ('Adj Close', 'AAA.PA') |   ('Adj Close', 'AALB.AS') |   ('Adj Close', 'AB.PA') |   ('Adj Close', 'ABCA.PA') |   ('Adj Close', 'ABEO.PA') |   ('Adj Close', 'ABI.BR') |   ('Adj Close', 'ABN.AS') |   ('Adj Close', 'ABNX.PA') |   ('Adj Close', 'ABO.BR') |   ('Adj Close', 'ABT.MI') |   ('Adj Close', 'ABVX.PA') |   ('Adj Close', 'AC.PA') |   ('Adj Close', 'AC5.MI') |   ('Adj Close', 'ACA.PA') |   ('Adj Close', 'ACAN.PA') |   ('Adj Close', 'ACCB.BR') |   ('Adj Close', 'ACE.MI') |   ('Adj Close', 'ACKB.BR') |   ('Adj Close', 'AD.AS') |   ('Adj Close', 'ADB.MI') |   ('Adj Close', 'ADOC.PA') |   ('Adj Close', 'ADP.PA') |   ('Adj Close', 'ADYEN.AS') |   ('Adj Close', 'AED.BR') |   ('Adj Close', 'AEDES.MI') |   ('Adj Close', 'AEF.MI') |   ('Adj Close', 'AELIS.PA') |   ('Adj Close', 'AF.PA') |   ('Adj Close', 'AFME.PA') |   ('Adj Close', 'AGFB.BR') |   ('Adj Close', 'AGN.AS') |   ('Adj Close', 'AGS.BR') |   ('Adj Clo

In [7]:
last_index = df_download_past_filled.index[-1]
last_download_past_date = last_index.date()
print(f'last download date: {last_download_past_date}')

download_missing_days = days_difference(today, last_download_past_date)
print(f'download missing days: {download_missing_days}')

# if download_missing_days > 1:
#     raise ValueError("Too many download missing days")


last download date: 2024-07-15
download missing days: 1


Download today's data

In [8]:
bourso_tickers = df_tickers['bourso'].iloc[1:872].values.tolist()

df_download_today = bourso.get_theoretical_opening_prices(bourso_tickers)

print(df_download_today)

Error fetching nan: could not convert string to float: '10.4542EUR'
Error fetching nan: could not convert string to float: '10.4542EUR'
Error fetching nan: could not convert string to float: '10.4542EUR'
Error fetching nan: could not convert string to float: '10.4542EUR'
Error fetching nan: could not convert string to float: '10.4542EUR'
Error fetching nan: could not convert string to float: '10.4542EUR'
Error fetching nan: could not convert string to float: '10.4542EUR'
Error fetching nan: could not convert string to float: '10.4542EUR'
Error fetching nan: could not convert string to float: '10.4542EUR'
Error fetching nan: could not convert string to float: '10.4542EUR'
Error fetching nan: could not convert string to float: '10.4542EUR'
Error fetching nan: could not convert string to float: '10.4542EUR'
Error fetching nan: could not convert string to float: '10.4542EUR'
Error fetching nan: could not convert string to float: '10.4542EUR'
Error fetching nan: could not convert string to 

Transform past data

In [9]:
def get_single_level_df(df, ohlcv):
    new_df = df[[ohlcv]]
    new_df = hf.remove_top_column_name(new_df)

    return new_df

def get_ohlcv_dfs(df):
    df_open = get_single_level_df(df, 'Open')
    df_high = get_single_level_df(df, 'High')
    df_low = get_single_level_df(df, 'Low')
    df_close = get_single_level_df(df, 'Close')
    df_volume = get_single_level_df(df, 'Volume')
    
    return {'df_open': df_open, 'df_high': df_high, 'df_low': df_low,
            'df_close': df_close, 'df_volume': df_volume}

In [10]:
list_dfs_ohlcv = get_ohlcv_dfs(df_download_past_filled)
df_buy = get_single_level_df(df_download_past_filled, buy_time)

print(df_buy.tail(3).to_markdown())

| Date                |   A2A.MI |   A5G.IR |   AAA.PA |   AALB.AS |   AB.PA |   ABCA.PA |   ABEO.PA |   ABI.BR |   ABN.AS |   ABNX.PA |   ABO.BR |   ABT.MI |   ABVX.PA |   AC.PA |   AC5.MI |   ACA.PA |   ACAN.PA |   ACCB.BR |   ACE.MI |   ACKB.BR |   AD.AS |   ADB.MI |   ADOC.PA |   ADP.PA |   ADYEN.AS |   AED.BR |   AEDES.MI |   AEF.MI |   AELIS.PA |   AF.PA |   AFME.PA |   AGFB.BR |   AGN.AS |   AGS.BR |   AI.PA |   AIR.PA |   AJAX.AS |   AKE.PA |   AKOM.PA |   AKW.PA |   AKZA.AS |   ALFEN.AS |   ALK.MI |   ALLFG.AS |   ALO.PA |   ALTA.PA |   ALTR.LS |   ALW.MI |   ALX.AS |   AM.PA |   AMG.AS |   AMP.MI |   AMPLI.PA |   AMUN.PA |   AMUND.AS |   ANIM.MI |   ANTIN.PA |   APAM.AS |   ARAMI.PA |   ARCAD.AS |   AREIT.PA |   ARG.PA |   ARGX.BR |   ARIS.MI |   ARN.MI |   ARTE.PA |   ARTO.PA |   ARVBS.PA |   ARVEN.PA |   ASC.MI |   ASCE.BR |   ASM.AS |   ASML.AS |   ASRNL.AS |   ASY.PA |   ATE.PA |   ATEB.BR |   ATEME.PA |   ATLD.PA |   ATO.PA |   AUB.PA |   AUGR.PA |   AURE.PA |   AUTME.MI

Transform today's data

In [11]:
mapping_dict = dict(zip(df_tickers['bourso'], df_tickers['yahoo']))

df_download_today['ticker'] = df_download_today['ticker'].map(mapping_dict)

print(df_download_today.tail(5))


       ticker  theoretical_opening_price
864  VLTSA.PA                      9.710
865     VU.PA                    141.600
866     MF.PA                     84.800
867    WLN.PA                     10.345
868   XFAB.PA                      6.510


In [12]:
df_buy_today = df_download_today.set_index('ticker').T
df_buy_today.index = ['Date']

today_date = datetime.now().strftime("%Y-%m-%d 00:00:00")
df_buy_today.index = [datetime.strptime(today_date, "%Y-%m-%d %H:%M:%S")]
df_buy_today = df_buy_today.reindex(sorted(df_buy_today.columns), axis=1)

print(df_buy_today)

ValueError: cannot reindex on an axis with duplicate labels

In [20]:
print(df_buy_today.shape[1])
duplicates = df_download_today[df_download_today['ticker'].duplicated()]

if not duplicates.empty:
    print("Duplicate tickers found:")
    print(duplicates)
else:
    print("No duplicate tickers found.")
    
print(df_buy_today.to_markdown())

869
Duplicate tickers found:
      ticker  theoretical_opening_price
153   RYA.IR                        NaN
154   RYA.IR                        NaN
155   RYA.IR                        NaN
156   RYA.IR                        NaN
157   RYA.IR                        NaN
158   RYA.IR                        NaN
159   RYA.IR                        NaN
160   RYA.IR                        NaN
161   RYA.IR                        NaN
162   RYA.IR                        NaN
163   RYA.IR                        NaN
164   RYA.IR                        NaN
165   RYA.IR                        NaN
166   RYA.IR                        NaN
167   RYA.IR                        NaN
727  ADOC.PA                       39.6
|                     |   AGN.AS |   AJAX.AS |   AMUND.AS |   ALX.AS |   AZRN.AS |   AZRNW.AS |   BSGR.AS |   BNJ.AS |   BNJW.AS |   BESI.AS |   BAI.AS |   BAIW.AS |   BGHL.AS |   BRNL.AS |   CABKA.AS |   CMCOM.AS |   CRBN.AS |   CTAC.AS |   CVC.AS |   DGB.AS |   EAS2P.AS |   EBUS.AS |   EH

Transform: Create inputs

In [ ]:
df_inputs = inputs.get_inputs(df_buy, list_dfs_ohlcv)

last_day_index = df_inputs.index.get_level_values(0)[-1] # Get the last value in the top-level index
df_inputs_last_day = df_inputs.loc[last_day_index]

X_inputs_last_day = df_inputs_last_day.values # Convert DataFrame to numpy array

print(df_inputs_last_day.tail(3).to_markdown())

In [ ]:
print(f'shape of X_inputs_last_day (rows -> tested data, columns -> inputs): {X_inputs_last_day.shape}')

In [ ]:
scaler = hf.load_object(scaler_path)

X_inputs_last_day_scaled = scaler.transform(X_inputs_last_day)

Load model

In [ ]:
model = tf.keras.models.load_model(model_path)

Predict

In [ ]:
prediction = model.predict(X_inputs_last_day_scaled)
df_prediction = pd.DataFrame(prediction, columns=['buy', 'do_not_buy'], index=df_inputs_last_day.index)
# df_prediction = df_prediction['buy']
df_prediction = df_prediction.sort_values(by='do_not_buy')


print(df_prediction.to_markdown())


Evaluate

In [ ]:
df_open = list_dfs_ohlcv['df_open']
df_close = list_dfs_ohlcv['df_close']

df_open_last_day = df_open.loc[last_day_index]
df_close_last_day = df_close.loc[last_day_index]

df_real = pd.DataFrame()
df_real['variation'] = df_close_last_day / df_open_last_day

In [ ]:
df_low = list_dfs_ohlcv['df_low']
df_low_last_day = df_low.loc[last_day_index]
df_real['lowest_variation'] = df_low_last_day / df_open_last_day

df_real['profit'] = df_real['variation']
df_real.loc[df_real['lowest_variation'] < loss_limit, 'profit'] = loss_limit
df_real['profit'] *= hf.get_fee_coef(fee)

In [ ]:
df_evaluation = pd.concat([df_prediction, df_real], axis='columns')
print(df_evaluation)

In [ ]:
df_filtered_evaluation = df_evaluation[df_evaluation['buy'] > cumulated_probs_target]

count = df_filtered_evaluation['profit'].count()
average = df_filtered_evaluation['profit'].mean()
median = df_filtered_evaluation['profit'].median()

print(f'count: {count}')
print(f'average profit: {average}')
print(f'median profit: {median}')

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 7))
plt.scatter(x=df_filtered_evaluation['buy'], y=df_filtered_evaluation['variation'], color='blue', alpha=0.3, s=5)
plt.title('variation vs buy prediction')
plt.xlabel('buy prediction')
plt.ylabel('real variation')
plt.grid(True)
plt.show()